# HoughLines Overlay

In [1]:
from webcam import WebcamVideoStream, FPS
import cv2
import numpy as np
import time
from hough import houghOverlay

# loading the Overlay
hw = houghOverlay('hwvhough2.bit')

Configure and load the webcam.

In [2]:
# Some parameters to perform the simulation

camera = 0
iterations = 100

print("[INFO] sampling THREADED frames from webcam")
vs = WebcamVideoStream(hw.frame, src=camera).start()

[INFO] sampling THREADED frames from webcam


In [3]:
# Hardware

print('Hardware execution...')
fps = FPS().start()

while fps._numFrames < iterations:
    [lines, segments] = hw.HoughLines(20,30,80,5,30)
    fps.update()
    
fps.stop()
print("[INFO] elapsed time: {:.2f}".format(fps.elapsed()))
print("[INFO] approx. FPS: {:.2f}".format(fps.fps()))

Hardware execution...
[INFO] elapsed time: 0.66
[INFO] approx. FPS: 152.51


In [4]:
# Software

frame = np.zeros((480, 640, 3), np.uint8)
vs.frame = frame
fps = FPS().start()
print('Software execution...')

def sw_houghlines(sw_frame):
    gray = cv2.cvtColor(sw_frame, cv2.COLOR_BGR2GRAY)
    blur = cv2.blur(gray, (3,3))
    edges = cv2.Canny(blur, 20, 30, apertureSize = 3)
    lines = cv2.HoughLines(edges, 1, np.pi/90, 80)
    segments = cv2.HoughLinesP(edges, 1, np.pi/90, 80, 30, 5)
    return [lines, segments]

# DIP loop pipeline
while fps._numFrames < iterations:
    sw_frame = vs.read()
    [lines, segments] = sw_houghlines(sw_frame)
    fps.update()
    
fps.stop()
print("[INFO] elapsed time: {:.2f}".format(fps.elapsed()))
print("[INFO] approx. FPS: {:.2f}".format(fps.fps()))

Software execution...
[INFO] elapsed time: 42.58
[INFO] approx. FPS: 2.35


Turn off the camera.

In [5]:
vs.stop()